In [1]:
from data import get_data
from model import Net
from utils import train, test
import torch
import torch.optim as optim


In [2]:
train_data = get_data(train=True)
test_data = get_data(train=False)

100%|██████████| 9912422/9912422 [00:05<00:00, 1761287.21it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 6476113.01it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 1690209.58it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2175462.92it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

CUDA Available? False


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

dropout_value = 0.1
model = Net(dropout_value=dropout_value).to(device)
summary(model, input_size=(1, 28, 28))

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 64, 7, 7]           2,112
      BatchNorm2d-10             [-1, 64, 7, 7]             128
        MaxPool2d-11             [-1, 64, 3, 3]               0
           Conv2d-12             [-1, 10, 1, 1]           5,770
Total params: 12,906
Trainable params: 

In [7]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []

dropout_value = 0.1
model =  Net(dropout_value).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch, train_losses, train_acc)
    # scheduler.step()
    test(model, device, test_loader, test_losses, test_acc)

EPOCH: 0


Loss=0.14021892845630646 Batch_id=937 Accuracy=95.14: 100%|██████████| 938/938 [02:15<00:00,  6.94it/s]  



Test set: Average loss: 0.0656, Accuracy: 9790/10000 (97.90%)

EPOCH: 1


Loss=0.1594468504190445 Batch_id=272 Accuracy=97.52:  29%|██▉       | 273/938 [00:45<01:51,  5.96it/s]   


KeyboardInterrupt: 